In [ ]:
!pip install gymnasium numpy


In [ ]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np

class PuzzleEnv(gym.Env):
    def __init__(self):
        super(PuzzleEnv, self).__init__()
        self.rows = 4  # filas
        self.cols = 5  # columnas
        self.state = np.arange(self.rows * self.cols).reshape(self.rows, self.cols)  # estado actual de la columna
        self.goal_state = np.copy(self.state)  # estado ordenado
        self.action_space = spaces.Discrete(4)  # up, down, left, right
        # define el espacio de observacion que es una matriz de 4x5 entre valores de 0 y 19
        self.observation_space = spaces.Box(low=0, high=self.rows*self.cols-1, shape=(self.rows, self.cols), dtype=int)

    # restablece el entrono a un estado inicial
    def reset(self):
        self.state = np.arange(self.rows * self.cols).reshape(self.rows, self.cols)
        np.random.shuffle(self.state.flat)  # mezcla los elementos del rompecabezas
        return self.state

    def step(self, action):
        row, col = np.argwhere(self.state == 0)[0]  # encuentra la posicion del espacio vacio
        if action == 0 and row > 0:  # up mueve arriba si no esta en la primera fila
            self.state[row, col], self.state[row - 1, col] = self.state[row - 1, col], self.state[row, col]
        elif action == 1 and row < self.rows - 1:  # down abajo
            self.state[row, col], self.state[row + 1, col] = self.state[row + 1, col], self.state[row, col]
        elif action == 2 and col > 0:  # left izquierda
            self.state[row, col], self.state[row, col - 1] = self.state[row, col - 1], self.state[row, col]
        elif action == 3 and col < self.cols - 1:  # right derecha
            self.state[row, col], self.state[row, col + 1] = self.state[row, col + 1], self.state[row, col]

        # devuelve el nuevo estado
        done = np.array_equal(self.state, self.goal_state)
        reward = 1 if done else -0.1

        return self.state, reward, done, {}

    # muestra el estado actual del rompecabezas
    def render(self):
        print(self.state)


In [39]:
# Create the environment
env = PuzzleEnv()

# Define the Q-learning parameters
alpha = 0.1  # learning rate
gamma = 0.99  # discount factor
epsilon = 1.0  # exploration rate
epsilon_decay = 0.995
min_epsilon = 0.01
episodes = 1

# Initialize the Q-table con zeros
q_table = np.zeros((env.rows * env.cols, env.action_space.n))

# Obtiene el estado actual
def get_state(env):
    return np.argmax(env.state.flatten())

# Elige una accion basandose en la tasa de exploracion
def choose_action(state):
    if np.random.rand() < epsilon:
        return np.random.choice(env.action_space.n)
    else:
        return np.argmax(q_table[state])

# Entrenamos al agente
for episode in range(episodes):  # itera en los episodios
    env.reset()
    state = get_state(env)
    done = False

    while not done:
        action = choose_action(state)  # se elige utilizando la function choose
        next_state, reward, done, _ = env.step(action)  # recibe
        next_state = get_state(env)

        # valor maximo de q
        q_value = q_table[state, action]
        max_next_q_value = np.max(q_table[next_state])

        # metodo incremental
        q_table[state, action] = q_value + alpha * (reward + gamma * max_next_q_value - q_value)

        state = next_state

    # reduccion de epsilon reduce la exploracion
    epsilon = max(min_epsilon, epsilon * epsilon_decay)

# Vemos como el agente entreno
env.reset()
state = get_state(env)
done = False

while not done:
    env.render()  # se muestra el estado actual del rompecabezas
    action = choose_action(state)  # se utiliza choose
    state, _, done, _ = env.step(action)  # se realiza la accion y se obtiene el siguiente estado
    state = get_state(env)

<ipython-input-26-e2cfa2f0292b>:19: UserWarning: you are shuffling a 'flatiter' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(self.state.flat)  # mezcla los elementos del rompecabezas


KeyboardInterrupt: 